**From the documentation**

http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('./datasets/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lr_model = lr.fit(training)

In [8]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lr_model.intercept

0.14228558260358093

In [11]:
training_summary = lr_model.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

Split train and test

In [14]:
all_data = spark.read.format('libsvm').load('./datasets/sample_linear_regression_data.txt')

In [15]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [21]:
train_data.count(), test_data.count()

(341, 160)

In [22]:
correct_model = lr.fit(train_data)

In [25]:
test_results = correct_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -27.43785596399608|
|-26.674050930690253|
|-24.082371203716093|
| -22.08142339351675|
|-19.569317582220293|
|-20.066374004638263|
|-20.885929525673586|
| -19.47004560292366|
|-16.441458569986633|
|-19.922103526695587|
|-15.857252950317893|
| -14.17500041228667|
|-16.041943683969777|
|-13.262962200450605|
|-12.101586048396099|
|  -9.83991721075817|
|-12.210772550442055|
| -13.28748004831059|
|-10.211219212207203|
|-12.338335562709895|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.617060065948424

In [29]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = correct_model.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -1.133622905747348|
|(10,[0,1,2,3,4,5,...|-0.1314324977928179|
|(10,[0,1,2,3,4,5,...| 0.5714871133931222|
|(10,[0,1,2,3,4,5,...| 0.6490356293509456|
|(10,[0,1,2,3,4,5,...| -0.315243192053133|
|(10,[0,1,2,3,4,5,...| 0.3990553892665425|
|(10,[0,1,2,3,4,5,...| 2.0400070527750036|
|(10,[0,1,2,3,4,5,...| 1.1948320369190275|
|(10,[0,1,2,3,4,5,...|-0.8852621626893139|
|(10,[0,1,2,3,4,5,...|  3.229896505384482|
|(10,[0,1,2,3,4,5,...|0.13373733726932352|
|(10,[0,1,2,3,4,5,...|-1.2008573110256273|
|(10,[0,1,2,3,4,5,...| 0.9854607094273444|
|(10,[0,1,2,3,4,5,...| 0.2230341363459905|
|(10,[0,1,2,3,4,5,...| 0.1965991457209843|
|(10,[0,1,2,3,4,5,...|-2.0174331546712563|
|(10,[0,1,2,3,4,5,...| 0.5949972854264293|
|(10,[0,1,2,3,4,5,...| 1.9590641115328074|
|(10,[0,1,2,3,4,5,...|-0.7347004455757294|
|(10,[0,1,2,3,4,5,...| 1.7382052208008623|
+----------

**Real dataset**

In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [38]:
from pyspark.ml.regression import LinearRegression

In [39]:
data = spark.read.csv('./datasets/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [40]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [42]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [44]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [45]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [46]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

In [47]:
output = assembler.transform(data)

In [50]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [49]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [51]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [52]:
final_data = output.select('features', 'Yearly Amount Spent')

In [53]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [54]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [55]:
train_data.count(), test_data.count()

(361, 139)

In [56]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [57]:
lr_model = lr.fit(train_data)

In [58]:
test_results = lr_model.evaluate(test_data)

In [59]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.930487884527906|
|-3.6852603379335847|
|  3.303431312894304|
|-4.1929138215848525|
|  -6.85455130040873|
| -5.987042421372848|
| 18.031493535886966|
|-1.0054421979292556|
| -8.440370425149808|
|  2.190000067546009|
| -5.420803449854077|
|-2.0163447822571356|
|-2.7088155302626546|
| -6.313096363392731|
|-14.034373970013348|
| -6.316525046521178|
|  5.293288144879057|
|  7.656180171210167|
|-18.123140197450766|
|-0.4135966923532237|
+-------------------+
only showing top 20 rows



In [60]:
test_results.rootMeanSquaredError

9.041798472442235

In [61]:
test_results.r2

0.9877006347570129

In [62]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [64]:
unlabeled_data = test_data.select('features')

In [65]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[30.8364326747734...|
|[31.3662121671876...|
|[31.4252268808548...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8186165667690...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.9365486184489...|
|[31.9453957483445...|
|[31.9480174211613...|
|[31.9549038566348...|
|[31.9563005605233...|
|[32.0047530203648...|
+--------------------+
only showing top 20 rows



In [67]:
predictions = lr_model.transform(unlabeled_data)

In [69]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|  450.850254311702|
|[30.8364326747734...| 471.1871607649232|
|[31.3662121671876...| 427.2854512435906|
|[31.4252268808548...| 534.9596324763468|
|[31.5147378578019...|496.66703929687014|
|[31.5257524169682...|449.95266923125473|
|[31.6005122003032...|461.14135795520997|
|[31.6610498227460...| 417.3637957778301|
|[31.7207699002873...| 547.2153039031728|
|[31.7366356860502...| 494.7434461879859|
|[31.7656188210424...| 501.9748850854612|
|[31.8186165667690...| 448.4350181523928|
|[31.8530748017465...|461.99393899261463|
|[31.8745516945853...| 398.5983406096602|
|[31.9365486184489...|441.23375886534154|
|[31.9453957483445...| 663.3364489841731|
|[31.9480174211613...|456.62758874801875|
|[31.9549038566348...| 432.3416997687168|
|[31.9563005605233...| 565.2490719446496|
|[32.0047530203648...|464.15957781298266|
+--------------------+------------